In [39]:
from bs4 import BeautifulSoup
import requests
import codecs
import re
import time
import pandas as pd

url = 'http://www.seed-db.com/companies/funding?value=1000000' # get the website we'll scrape
companies = BeautifulSoup(requests.get(url).text, 'lxml') # tranform it to 'beautifulsoup' type

In [ ]:
# access the table with target data
table = companies.find('tbody')
rows = table.find_all('tr')

In [ ]:
# create a list to store data
final_result = []

# define the function to get data about seed accelerators with corresponding links
def accelerator_info(link, startup):
    accel_info = BeautifulSoup(requests.get(link).text, 'lxml')
    body = accel_info.find('body')
    
    introduction = body.find_all('div', {'class': 'container-fluid'})[1].find('div', {'class': 'col-md-12'})
    
    basic_info = introduction.find_all('div', {'class': 'row'})[0].find('div', {'class': 'col-md-4'})
    startup['accel_website'] = basic_info.find('a')['href']
    startup['accel_estb_yr'] = re.findall('[0-9]+', basic_info.find_all('p')[1].text)
    startup['accel_add'] = basic_info.find_all('p')[2].text
    
    funded_info = introduction.find_all('div', {'class': 'row'})[1].find_all('div', {'class': 'col-md-6'})[1]
    
    startup['accel_num_funded'] = re.findall('[0-9]+', funded_info.find_all('p')[4].text.replace(',', ''))
    startup['accel_num_exited'] = re.findall('[0-9]+', funded_info.find_all('p')[5].text.replace(',', ''))
    startup['accel_amt_exited'] = re.findall('[0-9]+', funded_info.find_all('p')[6].text.replace(',', ''))
    startup['accel_amt_funded'] = re.findall('[0-9]+', funded_info.find_all('p')[7].text.replace(',', ''))
   
    return startup

# read data row by row from the website
i = 0
for row in rows:
    try:
        company = {} # create a dictionary to store information of every row
        tds = row.find_all('td') # search for the data

        # extract information form the web
        state = tds[0].text.replace('\n', '')
        name = tds[1].text.replace('\n', '')
        website = tds[2].find('a')['href']
        funding = tds[6].text.replace('$', '').replace(',', '').strip()
        rounds = tds[7].text
        accelerator = tds[3].text

        # get the information of corresponding accelerators 
        accel_link = 'http://seed-db.com/' + tds[3].find('a')['href']
        company = accelerator_info(accel_link, company)

        # store the result into the dictionary
        company['state'] = state
        company['name'] = name
        company['website'] = website
        company['seed_accelartor'] = accelerator
        company['funding'] = funding
        company['rounds'] = rounds
    
        # add information of every row as an element to the list
        final_result.append(company)
        i = i+1
        print '############### %d ############' % i
    
    except:
        print '********************** Error *****************'
        time.sleep(10) # restart
        



In [133]:
# store data into the file
f = codecs.open('companies.csv', 'w', 'utf-8') # some variables will be formatted with R
final_data = pd.DataFrame(final_result)
final_data.to_csv('companies.csv')
f.close()

'\nda\n'